The purpose of this notebook is to create a function that takes the imported player list:
- clean to only needed information
- seperate by pither and batter
- merge each on player name to get projections from model
- merge back pitcher and batter data 
- output df for now

Next steps:
- build line up based on salary

In [107]:
# imports
import pandas as pd
import numpy as np

In [2]:
# will need fanduel import
fd = pd.read_csv('../CapStone_Data/FanDuel-MLB-2021 ET-05 ET-04 ET-58318-players-list.csv')

In [3]:
# print first 5 rows of fd
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Probable Pitcher,Batting Order,Roster Position
0,58318-52859,P,Jacob,Jacob deGrom,deGrom,61.600000,5.0,12500,NYM@STL,NYM,STL,NaN,NaN,NaN,Yes,NaN,P
1,58318-16956,P,Gerrit,Gerrit Cole,Cole,54.166667,6.0,12200,HOU@NYY,NYY,HOU,NaN,NaN,NaN,NaN,0.0,P
2,58318-82554,P,Shane,Shane Bieber,Bieber,55.666667,6.0,12000,CLE@KC,CLE,KC,NaN,NaN,NaN,NaN,NaN,P
3,58318-5481,P,Max,Max Scherzer,Scherzer,42.166667,6.0,12000,ATL@WSH,WSH,ATL,NaN,NaN,NaN,NaN,0.0,P
4,58318-82604,P,Corbin,Corbin Burnes,Burnes,49.600000,5.0,11100,MIL@PHI,MIL,PHI,IL,Undisclosed,NaN,NaN,0.0,P


only columns needed:
- id - will need this later for template
- Position
- nickname - renamed to Name
- salary
- game
- team
- opponent
- injury indicator
- probable pitcher

In [4]:
fd.columns

Index(['Id', 'Position', 'First Name', 'Nickname', 'Last Name', 'FPPG',
       'Played', 'Salary', 'Game', 'Team', 'Opponent', 'Injury Indicator',
       'Injury Details', 'Tier', 'Probable Pitcher', 'Batting Order',
       'Roster Position'],
      dtype='object')

In [5]:
# clean up fd to match column list above
fd.drop(columns=['First Name', 'Last Name', 'FPPG', 'Played',
                 'Injury Details', 'Tier', 'Batting Order', 'Roster Position'], inplace=True)

Next step is fill nulls in probable pitcher and injury indicator

In [6]:
# filling nulls for probable pitcher
fd['Probable Pitcher'].fillna('No', inplace=True)

In [7]:
# fill nulls for injury indicator
fd['Injury Indicator'].fillna('Healthy', inplace=True)

In [8]:
# review new cleaned df
fd.head()

,Id,Position,Nickname,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
0,58318-52859,P,Jacob deGrom,12500,NYM@STL,NYM,STL,Healthy,Yes
1,58318-16956,P,Gerrit Cole,12200,HOU@NYY,NYY,HOU,Healthy,No
2,58318-82554,P,Shane Bieber,12000,CLE@KC,CLE,KC,Healthy,No
3,58318-5481,P,Max Scherzer,12000,ATL@WSH,WSH,ATL,Healthy,No
4,58318-82604,P,Corbin Burnes,11100,MIL@PHI,MIL,PHI,IL,No


In [9]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                831 non-null    object
 1   Position          831 non-null    object
 2   Nickname          831 non-null    object
 3   Salary            831 non-null    int64 
 4   Game              831 non-null    object
 5   Team              831 non-null    object
 6   Opponent          831 non-null    object
 7   Injury Indicator  831 non-null    object
 8   Probable Pitcher  831 non-null    object
dtypes: int64(1), object(8)
memory usage: 58.6+ KB


Next step rename Nickname to name

In [10]:
# renaming nickname column
fd.rename(columns={'Nickname': 'Name'}, inplace=True)

Next step filter to only healthy players

In [11]:
# fitler to only healthy players
fd = fd.loc[fd['Injury Indicator'] == 'Healthy']

In [12]:
# review dataframe
fd.head()

,Id,Position,Name,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
0,58318-52859,P,Jacob deGrom,12500,NYM@STL,NYM,STL,Healthy,Yes
1,58318-16956,P,Gerrit Cole,12200,HOU@NYY,NYY,HOU,Healthy,No
2,58318-82554,P,Shane Bieber,12000,CLE@KC,CLE,KC,Healthy,No
3,58318-5481,P,Max Scherzer,12000,ATL@WSH,WSH,ATL,Healthy,No
5,58318-16931,P,Yu Darvish,11000,PIT@SD,SD,PIT,Healthy,No


In [13]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681 entries, 0 to 830
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                681 non-null    object
 1   Position          681 non-null    object
 2   Name              681 non-null    object
 3   Salary            681 non-null    int64 
 4   Game              681 non-null    object
 5   Team              681 non-null    object
 6   Opponent          681 non-null    object
 7   Injury Indicator  681 non-null    object
 8   Probable Pitcher  681 non-null    object
dtypes: int64(1), object(8)
memory usage: 53.2+ KB


next step is to split into pithcers and batters

In [14]:
# split using .loc by position and make new dataframe for pitchers
pitchers = fd.loc[fd['Position']=='P']

In [15]:
# split using .loc by position and make new dataframe for batters
batters = fd.loc[fd['Position']!='P']

Now we have cleaned dataframes for each group. Now lets focus on pitchers to get projections added to dataframe.

First step for pitchers is to get the probable pitchers first, we only care about the ones that will start.

In [38]:
# save pitchers df to only starting pitchers
pitchers = pitchers.loc[pitchers['Probable Pitcher']=='Yes']

In [39]:
# review changes
pitchers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 398
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                21 non-null     object
 1   Position          21 non-null     object
 2   Name              21 non-null     object
 3   Salary            21 non-null     int64 
 4   Game              21 non-null     object
 5   Team              21 non-null     object
 6   Opponent          21 non-null     object
 7   Injury Indicator  21 non-null     object
 8   Probable Pitcher  21 non-null     object
dtypes: int64(1), object(8)
memory usage: 1.6+ KB


Next step is to combine projections with 2021 stats.
- import testing data with projections from model
- merge the two data frames

In [40]:
# read in pitcher projections
pitcher_proj = pd.read_csv('../Projections/pitcher_projections_2021.csv')

Next step merge.

In [41]:
# merge attempt
pitcher_projections = pitchers.merge(pitcher_proj, how='left', on='Name')

Next step is to drop nulls from this list.  Will need to handle this in previous processes for final product.

In [42]:
pitcher_projections.dropna(inplace=True)

Next step is to clean up and only take what is needed from pitchers so that it can be merged with batters.
Columns needed:
- ID
- Position
- Name
- Salary
- Team
- Opponent
- Proj_FPPG

In [43]:
# overwrite df with only the columns needed
pitcher_projections = pitcher_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent', 'Proj_FPPG']]

In [44]:
# rename team column
pitcher_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

Now we have a cleaned and organized df of our pitchers that are starting. Next is the batters.

---

First step is to merge on projections.

In [47]:
# read in projections file
batter_21 = pd.read_csv('../Projections/batter_projections_2021.csv')

In [50]:
# merge projections with batter df, creating new df
batters_projections = batters.merge(batter_21, how='left', on='Name')

Will have to drop nulls for now, but need to figure out why there is no data for those batters.

In [54]:
# dropping batters with no projections
batters_projections.dropna(inplace=True)

In [64]:
# drop unneeded columns for merge with pitcher, overwrite current df
batters_projections = batters_projections[['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Proj_FPPG']]

Now there are two clean dataframes with the same columns, now we will determine optimal line up.

Steps to get optimal line up:
1. set salary cap
2. sort by fppg

    a. choose pitcher - based on projected fppg; in final product need to incoporate stats with strong coefficient values
    b. choose batters based on remaining salary and highest fppg; in final product need to incoporate stats with strong coefficient values
    
3. create list to append choices to
4. save to one df for output
5. take df and save to templet for export

In [68]:
# set cap for fanduel
salary_cap = 35_000

In [81]:
# sort pitcher by fppg projections
pitcher_projections.sort_values(by='Proj_FPPG', ascending=False, inplace=True, ignore_index=True)

In [102]:
# create a player list to 
lineup = []
lineup.append(pitcher_projections.values[0])

In [86]:
# need to update remaining salary
salary_cap -= pitcher_projections['Salary'][0]

In [96]:
# with updated salary fill remaining roster based on position and highest fppg
# create position list for remaining roster spots
position_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'OF', 'OF']
# sort batters by FPPG
batters_projections.sort_values(by='Proj_FPPG', ascending=False, inplace=True, ignore_index=True)

<ipython-input-96-36e149c8ef92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters_projections.sort_values(by='Proj_FPPG', ascending=False, inplace=True, ignore_index=True)


In [111]:
# create for loop for each position in list to take highest fppg
for pos in position_list:
    # setting counter to increase if player is already in list
    # this is inside the for loop beacuse it needs to be per position
    counter = 0
    print(batters_projections.loc[batters_projections['Position'] == pos].values[counter])
    
    #lineup.append(batters_projections.loc[batters_projections['Position'] == pos].values[counter])

['58318-79174' 'C' 'Jacob Nottingham' 2500 'MIL' 'PHI' 23.84243805551742]
['58318-104483' '1B' 'Jared Walsh' 3500 'LAA' 'TB' 9.26517900554932]
['58318-68587' '2B' 'Ozzie Albies' 3300 'ATL' 'WSH' 7.6460846577038115]
['58318-5104' '3B' 'Pablo Sandoval' 2000 'ATL' 'WSH' 8.927933337580752]
['58318-79393' 'SS' 'Fernando Tatis Jr.' 4400 'SD' 'PIT' 9.31301982881544]
['58318-12933' 'OF' 'Mike Trout' 4500 'LAA' 'TB' 11.491166229732436]
['58318-12933' 'OF' 'Mike Trout' 4500 'LAA' 'TB' 11.491166229732436]
['58318-12933' 'OF' 'Mike Trout' 4500 'LAA' 'TB' 11.491166229732436]


In [121]:
type(batters_projections.loc[batters_projections['Position'] == 'OF']['Name'].values[0])

str

In [119]:
lineup

[array(['58318-79951', 'P', 'Nick Pivetta', 8300, 'BOS', 'DET',
        27.943867072751747], dtype=object),
 array(['58318-79174', 'C', 'Jacob Nottingham', 2500, 'MIL', 'PHI',
        23.84243805551742], dtype=object),
 array(['58318-104483', '1B', 'Jared Walsh', 3500, 'LAA', 'TB',
        9.26517900554932], dtype=object),
 array(['58318-68587', '2B', 'Ozzie Albies', 3300, 'ATL', 'WSH',
        7.6460846577038115], dtype=object),
 array(['58318-5104', '3B', 'Pablo Sandoval', 2000, 'ATL', 'WSH',
        8.927933337580752], dtype=object),
 array(['58318-79393', 'SS', 'Fernando Tatis Jr.', 4400, 'SD', 'PIT',
        9.31301982881544], dtype=object),
 array(['58318-12933', 'OF', 'Mike Trout', 4500, 'LAA', 'TB',
        11.491166229732436], dtype=object),
 array(['58318-12933', 'OF', 'Mike Trout', 4500, 'LAA', 'TB',
        11.491166229732436], dtype=object),
 array(['58318-12933', 'OF', 'Mike Trout', 4500, 'LAA', 'TB',
        11.491166229732436], dtype=object)]

In [131]:
if batters_projections.loc[batters_projections['Position'] == 'OF']['Name'].values[0] not in lineup[0]:
    print('y')

y


In [130]:
lineup

[array(['58318-79951', 'P', 'Nick Pivetta', 8300, 'BOS', 'DET',
        27.943867072751747], dtype=object),
 array(['58318-79174', 'C', 'Jacob Nottingham', 2500, 'MIL', 'PHI',
        23.84243805551742], dtype=object),
 array(['58318-104483', '1B', 'Jared Walsh', 3500, 'LAA', 'TB',
        9.26517900554932], dtype=object),
 array(['58318-68587', '2B', 'Ozzie Albies', 3300, 'ATL', 'WSH',
        7.6460846577038115], dtype=object),
 array(['58318-5104', '3B', 'Pablo Sandoval', 2000, 'ATL', 'WSH',
        8.927933337580752], dtype=object),
 array(['58318-79393', 'SS', 'Fernando Tatis Jr.', 4400, 'SD', 'PIT',
        9.31301982881544], dtype=object),
 array(['58318-12933', 'OF', 'Mike Trout', 4500, 'LAA', 'TB',
        11.491166229732436], dtype=object),
 array(['58318-12933', 'OF', 'Mike Trout', 4500, 'LAA', 'TB',
        11.491166229732436], dtype=object),
 array(['58318-12933', 'OF', 'Mike Trout', 4500, 'LAA', 'TB',
        11.491166229732436], dtype=object)]

In [169]:
batters_projections.loc[batters_projections['Position'] == 'OF'].index.values[0]
#batters_projections.drop([0])

2

In [148]:
batters_projections

,Id,Position,Name,Salary,Team,Opponent,Proj_FPPG
0,58318-79174,C,Jacob Nottingham,2500,MIL,PHI,23.842438
1,58318-102513,C,Mario Feliciano,2000,MIL,PHI,12.261206
2,58318-12933,OF,Mike Trout,4500,LAA,TB,11.491166
3,58318-39086,OF,Byron Buxton,4300,MIN,TEX,11.413326
4,58318-13540,OF,J.D. Martinez,4200,BOS,DET,10.688011
...,...,...,...,...,...,...,...
277,58318-80836,SS,Tzu-Wei Lin,2000,MIN,TEX,0.014872
278,58318-137070,C,Joseph Odom,2000,TB,LAA,0.005255
279,58318-60636,OF,Derek Hill,2000,DET,BOS,-0.011430
280,58318-149374,SS,Sean Kazmar Jr.,2000,ATL,WSH,-0.016745
